In [1]:
import pandas as pd
import psycopg2
from moveworks.utils.config.yaml_config import load_yaml_config_to_box


In [2]:
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successful..................")
        
    except OperationalError as err:
        print(color.BOLD + """Unable to connect to the DB, please double check your credentials file 
              and ensure you are connected to the VPN
        """)
        # passing exception to function     
        show_psycopg2_exception(err)   
        # set the connection to 'None' in case of error
        conn = None
    
    return conn

In [3]:
config = load_yaml_config_to_box('dse_app_db_config.yml')
        
conn_params_dic = {
     "host": config['DSE_APP_DB_HOST'],
    "port": config['DSE_APP_DB_PORT'],
    "user": config['DSE_APP_DB_USER'],
    "password": config['DSE_APP_DB_PWD'],
    "database": config['DSE_APP_DB_NAME']
}

connection = connect(conn_params_dic)

Connecting to the PostgreSQL...........
Connection successful..................


In [4]:
query = """
select distinct 
    ID.ORG,
    ID.reassignment_count_bin,
    AVG(calendar_duration) as pttr
FROM
    AIA_METRICS.ITSM_DATA ID
WHERE
    ID.EXCLUDE = 'No'
AND 
    ID.ticket_type = 'INC'
AND
    ID.reassignment_count_bin NOT IN ('No Reassignment Bin')
AND
    ID.reassignment_count_bin  IS NOT NULL
GROUP BY 
    ID.ORG,
    ID.reassignment_count_bin 
ORDER BY
    ID.ORG;
"""

ticket_data = pd.read_sql_query(query, con=connection)
ticket_data[['reassignment_count_bin', 'pttr']].groupby('reassignment_count_bin').mean()

,pttr
reassignment_count_bin,
0 Reassignments,3.516781
1 Reassignment,6.219913
2+ Reassignments,10.844113
